# D2E2S Model Training on Kaggle
## Dual-channel Enhanced Entity-Sentiment Model for Aspect-Based Sentiment Analysis

This notebook provides a complete setup for training the D2E2S model on Kaggle with optimal hyperparameters.

**Dataset**: 15res (Restaurant Reviews 2015)
**Optimal Configuration**: Hybrid GCN with production-ready hyperparameters

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install torch==2.4.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch-geometric==2.3.1
!pip install transformers==4.28.1
!pip install tensorboardX==2.6
!pip install scikit-learn==1.2.2
!pip install spacy>=3.7.2,<3.8.0
!pip install matplotlib==3.7.1
!pip install pydantic>=2.7.0
!pip install pandas>=1.3.0
!pip install seaborn>=0.11.0

# Download spacy model
!python -m spacy download en_core_web_sm

## Step 2: Setup Environment and Imports

In [ ]:
import os
import sys
import argparse
import math
import warnings
import json
from pathlib import Path

# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

import torch
import transformers
from torch.optim import optimizer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AdamW, AutoTokenizer, AutoConfig

warnings.filterwarnings("ignore")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## Step 3: Upload and Extract Dataset

**Instructions:**
1. Upload your D2E2S dataset as a ZIP file to Kaggle
2. The dataset should contain the following structure:
   - `data/15res/train_dep_triple_polarity_result.json`
   - `data/15res/test_dep_triple_polarity_result.json`
   - `data/types.json`
   - All model files in `models/` directory
   - All trainer files in `trainer/` directory

In [ ]:
# Extract dataset (adjust path according to your uploaded file)
import zipfile

# Assuming you uploaded the dataset as 'd2e2s_dataset.zip'
dataset_path = '/kaggle/input/d2e2s-dataset/d2e2s_dataset.zip'  # Adjust this path

# Extract dataset
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

# Verify extraction
print("Dataset extracted. Contents:")
for root, dirs, files in os.walk('/kaggle/working/'):
    level = root.replace('/kaggle/working/', '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files only
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

## Step 4: Define Optimal Configuration

Using the best hyperparameters from Optuna optimization for 15res dataset:

In [ ]:
# Optimal hyperparameters for 15res dataset (F1: 0.8644)
OPTIMAL_CONFIG = {
    "batch_size": 6,
    "lr": 0.000312,
    "lr_warmup": 0.12,
    "weight_decay": 0.0012,
    "gcn_type": "hybrid",
    "gcn_layers": 2,
    "attention_heads": 16,
    "hidden_dim": 1024,
    "gcn_dim": 768,
    "epochs": 40
}

print("Optimal Configuration:")
for key, value in OPTIMAL_CONFIG.items():
    print(f"  {key}: {value}")

## Step 5: Create Configuration Class

In [ ]:
class TrainingConfig:
    def __init__(self):
        # Dataset configuration
        self.dataset = "15res"
        self.dataset_file = {
            "train": "/kaggle/working/data/15res/train_dep_triple_polarity_result.json",
            "test": "/kaggle/working/data/15res/test_dep_triple_polarity_result.json",
            "types_path": "/kaggle/working/data/types.json"
        }
        
        # Optimal hyperparameters
        self.batch_size = OPTIMAL_CONFIG["batch_size"]
        self.lr = OPTIMAL_CONFIG["lr"]
        self.lr_warmup = OPTIMAL_CONFIG["lr_warmup"]
        self.weight_decay = OPTIMAL_CONFIG["weight_decay"]
        self.epochs = OPTIMAL_CONFIG["epochs"]
        self.attention_heads = OPTIMAL_CONFIG["attention_heads"]
        self.hidden_dim = OPTIMAL_CONFIG["hidden_dim"]
        self.gcn_dim = OPTIMAL_CONFIG["gcn_dim"]
        
        # GCN configuration
        self.gcn_type = OPTIMAL_CONFIG["gcn_type"]
        self.num_layers = OPTIMAL_CONFIG["gcn_layers"]
        self.use_improved_gcn = True
        self.use_residual = True
        self.use_layer_norm = True
        self.use_multi_scale = True
        self.use_graph_attention = True
        self.use_relative_position = True
        self.use_global_context = True
        
        # Model configuration
        self.pretrained_deberta_name = "microsoft/deberta-v2-xxlarge"
        self.tokenizer_path = "bert-base-uncased"
        self.max_span_size = 6
        self.max_pairs = 800
        self.sen_filter_threshold = 0.5
        
        # Training configuration
        self.seed = 42
        self.max_grad_norm = 1.0
        self.drop_out_rate = 0.3
        self.gcn_dropout = 0.1
        self.prop_drop = 0.05
        
        # Sampling configuration
        self.neg_entity_count = 50
        self.neg_triple_count = 50
        self.sampling_processes = 4
        self.sampling_limit = 100
        
        # Logging configuration
        self.log_path = "/kaggle/working/log/"
        self.train_log_iter = 1
        self.store_predictions = True
        self.store_examples = True
        self.example_count = None
        
        # Other configurations
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.cpu = False
        self.init_eval = False
        self.final_eval = False
        self.freeze_transformer = False
        self.lowercase = True
        self.is_bidirect = True
        self.use_gated = False
        
        # Additional dimensions
        self.emb_dim = 1536
        self.lstm_layers = 2
        self.lstm_dim = 384
        self.mem_dim = 768
        self.deberta_feature_dim = 1536
        self.size_embedding = 25
        self.span_generator = "Max"
        self.pooling = "avg"
        
        # Create necessary directories
        os.makedirs(self.log_path, exist_ok=True)
        
        self.label = self.dataset

# Create configuration
args = TrainingConfig()
print(f"Training on device: {args.device}")
print(f"Dataset: {args.dataset}")
print(f"Batch size: {args.batch_size}")
print(f"Learning rate: {args.lr}")
print(f"Epochs: {args.epochs}")

## Step 6: Import Model Components

**Note**: Make sure all model files are in the correct directory structure after extraction.

In [ ]:
# Add the working directory to Python path
sys.path.append('/kaggle/working')

# Import D2E2S components
try:
    from models.D2E2S_Model import D2E2SModel
    from models.General import set_seed
    from trainer import util, sampling
    from trainer.baseTrainer import BaseTrainer
    from trainer.entities import Dataset
    from trainer.evaluator import Evaluator
    from trainer.input_reader import JsonInputReader
    from trainer.loss import D2E2SLoss
    
    print("✓ All D2E2S components imported successfully")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure all model and trainer files are in the correct directories")

## Step 7: Define Training Class

In [ ]:
class D2E2S_Trainer(BaseTrainer):
    def __init__(self, args):
        super().__init__(args)
        self._tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xxlarge")
        self._predictions_path = os.path.join(
            self._log_path_predict, "predicted_%s_epoch_%s.json"
        )
        self._examples_path = os.path.join(
            self._log_path_predict, "sample_%s_%s_epoch_%s.html"
        )
        os.makedirs(self._log_path_result, exist_ok=True)
        os.makedirs(self._log_path_predict, exist_ok=True)
        self.max_pair_f1 = 40
        self.best_epoch = -1
        self.result_path = os.path.join(
            self._log_path_result, "result{}.txt".format(self.args.max_span_size)
        )

    def _preprocess(self, args, input_reader_cls, types_path, train_path, test_path):
        train_label, test_label = "train", "test"
        
        # Create log csv files
        self._init_train_logging(train_label)
        self._init_eval_logging(test_label)

        # Loading data
        input_reader = input_reader_cls(
            types_path,
            self._tokenizer,
            args.neg_entity_count,
            args.neg_triple_count,
            args.max_span_size,
        )
        input_reader.read({train_label: train_path, test_label: test_path})
        train_dataset = input_reader.get_dataset(train_label)

        # Preprocess
        train_sample_count = train_dataset.sentence_count
        updates_epoch = train_sample_count // args.batch_size
        updates_total = updates_epoch * args.epochs

        print(f"Dataset: {args.dataset}, Max span size: {args.max_span_size}")
        print(f"Training samples: {train_sample_count}")
        print(f"Updates per epoch: {updates_epoch}")
        print(f"Total updates: {updates_total}")
        
        return input_reader, updates_total, updates_epoch

    def train_model(self, train_path: str, test_path: str, types_path: str, input_reader_cls):
        args = self.args

        # Set seed
        set_seed(args.seed)

        train_label, test_label = "train", "test"
        input_reader, updates_total, updates_epoch = self._preprocess(
            args, input_reader_cls, types_path, train_path, test_path
        )
        train_dataset = input_reader.get_dataset(train_label)
        test_dataset = input_reader.get_dataset(test_label)

        # Load model
        config = AutoConfig.from_pretrained("microsoft/deberta-v2-xxlarge")

        model = D2E2SModel.from_pretrained(
            args.pretrained_deberta_name,
            config=config,
            cls_token=self._tokenizer.convert_tokens_to_ids("[CLS]"),
            sentiment_types=input_reader.sentiment_type_count - 1,
            entity_types=input_reader.entity_type_count,
            args=args,
        )
        model.to(args.device)
        
        print(f"Model loaded on {args.device}")
        print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

        # Create optimizer
        optimizer_params = self._get_optimizer_params(model)
        optimizer = AdamW(
            optimizer_params,
            lr=args.lr,
            weight_decay=args.weight_decay,
            correct_bias=False,
        )
        
        # Create scheduler
        scheduler = transformers.get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.lr_warmup * updates_total,
            num_training_steps=updates_total,
        )

        # Create loss function
        entity_criterion = torch.nn.CrossEntropyLoss(reduction="none")
        senti_criterion = torch.nn.BCEWithLogitsLoss(reduction="none")
        compute_loss = D2E2SLoss(
            senti_criterion,
            entity_criterion,
            model,
            optimizer,
            scheduler,
            args.max_grad_norm,
        )

        # Initial evaluation
        if args.init_eval:
            self._eval(model, test_dataset, input_reader, 0, updates_epoch)

        # Training loop
        print(f"\nStarting training for {args.epochs} epochs...")
        for epoch in range(args.epochs):
            print(f"\n=== Epoch {epoch + 1}/{args.epochs} ===")
            
            # Train epoch
            self.train_epoch(
                model, compute_loss, optimizer, train_dataset, updates_epoch, epoch
            )

            # Evaluate
            if not args.final_eval or (epoch == args.epochs - 1):
                self._eval(model, test_dataset, input_reader, epoch + 1, updates_epoch)
                
        print(f"\n🎉 Training completed!")
        print(f"Best F1 score: {self.max_pair_f1:.4f} at epoch {self.best_epoch}")
        
        return model

    def train_epoch(self, model, compute_loss, optimizer, dataset, updates_epoch, epoch):
        # Create data loader
        dataset.switch_mode(Dataset.TRAIN_MODE)
        data_loader = DataLoader(
            dataset,
            batch_size=self.args.batch_size,
            shuffle=True,
            drop_last=True,
            num_workers=self.args.sampling_processes,
            collate_fn=sampling.collate_fn_padding,
        )

        model.zero_grad()
        iteration = 0
        total = dataset.sentence_count // self.args.batch_size
        
        epoch_losses = []
        
        for batch in tqdm(data_loader, total=total, desc=f"Training epoch {epoch + 1}"):
            model.train()
            batch = util.to_device(batch, self.args.device)

            # Forward step
            entity_logits, senti_logits, batch_loss = model(
                encodings=batch["encodings"],
                context_masks=batch["context_masks"],
                entity_masks=batch["entity_masks"],
                entity_sizes=batch["entity_sizes"],
                sentiments=batch["rels"],
                senti_masks=batch["senti_masks"],
                adj=batch["adj"],
            )

            # Compute loss and optimize parameters
            epoch_loss = compute_loss.compute(
                entity_logits=entity_logits,
                senti_logits=senti_logits,
                batch_loss=batch_loss,
                senti_types=batch["senti_types"],
                entity_types=batch["entity_types"],
                entity_sample_masks=batch["entity_sample_masks"],
                senti_sample_masks=batch["senti_sample_masks"],
            )
            
            epoch_losses.append(epoch_loss)
            iteration += 1
            global_iteration = epoch * updates_epoch + iteration

            if global_iteration % self.args.train_log_iter == 0:
                self._log_train(
                    optimizer, epoch_loss, epoch, iteration, global_iteration, dataset.label
                )
        
        avg_loss = sum(epoch_losses) / len(epoch_losses) if epoch_losses else 0
        print(f"Average loss: {avg_loss:.4f}")
        return iteration

    def _eval(self, model, dataset, input_reader, epoch=0, updates_epoch=0, iteration=0):
        # Create evaluator
        evaluator = Evaluator(
            dataset,
            input_reader,
            self._tokenizer,
            self.args.sen_filter_threshold,
            self._predictions_path,
            self._examples_path,
            self.args.example_count,
            epoch,
            dataset.label,
        )
        
        # Create data loader
        dataset.switch_mode(Dataset.EVAL_MODE)
        data_loader = DataLoader(
            dataset,
            batch_size=self.args.batch_size,
            shuffle=False,
            drop_last=False,
            num_workers=self.args.sampling_processes,
            collate_fn=sampling.collate_fn_padding,
        )

        with torch.no_grad():
            model.eval()
            total = math.ceil(dataset.sentence_count / self.args.batch_size)
            
            for batch in tqdm(data_loader, total=total, desc=f"Evaluating epoch {epoch}"):
                batch = util.to_device(batch, self.args.device)

                # Run model (forward pass)
                result = model(
                    encodings=batch["encodings"],
                    context_masks=batch["context_masks"],
                    entity_masks=batch["entity_masks"],
                    entity_sizes=batch["entity_sizes"],
                    entity_spans=batch["entity_spans"],
                    entity_sample_masks=batch["entity_sample_masks"],
                    evaluate=True,
                    adj=batch["adj"],
                )
                entity_clf, senti_clf, rels = result
                evaluator.eval_batch(entity_clf, senti_clf, rels, batch)
                
            global_iteration = epoch * updates_epoch + iteration
            ner_eval, senti_eval, senti_nec_eval = evaluator.compute_scores()
            
            # Log results
            self._log_filter_file(ner_eval, senti_eval, evaluator, epoch)
            
        self._log_eval(
            *ner_eval, *senti_eval, *senti_nec_eval,
            epoch, iteration, global_iteration, dataset.label
        )
        
        # Print current results
        print(f"Sentiment F1: {senti_eval[2]:.4f}")
        if senti_eval[2] > self.max_pair_f1:
            print(f"🎯 New best F1 score: {senti_eval[2]:.4f}")

    def _log_filter_file(self, ner_eval, senti_eval, evaluator, epoch):
        f1 = float(senti_eval[2])
        if self.max_pair_f1 < f1:
            columns = ["mic_precision", "mic_recall", "mic_f1_score", "mac_precision", "mac_recall", "mac_f1_score"]
            
            senti_dic = {columns[i]: val for i, val in enumerate(senti_eval)}
            self.max_pair_f1 = f1
            self.best_epoch = epoch
            
            with open(self.result_path, mode="a", encoding="utf-8") as f:
                f.write(f"Epoch {epoch}:\n")
                f.write(f"Sentiment results: {senti_dic}\n\n")
                
            # Clean up old predictions
            try:
                for filename in os.listdir(self._log_path_predict):
                    os.remove(os.path.join(self._log_path_predict, filename))
            except Exception as e:
                print(f"Warning: Could not clean predictions directory: {e}")
                
            if self.args.store_predictions:
                evaluator.store_predictions()
            if self.args.store_examples:
                evaluator.store_examples()

    def _get_optimizer_params(self, model):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_params = [
            {
                "params": [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                "weight_decay": self.args.weight_decay,
            },
            {
                "params": [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        return optimizer_params

print("✓ D2E2S_Trainer class defined successfully")

## Step 8: Verify Dataset Files

In [ ]:
# Verify that all required files exist
required_files = [
    args.dataset_file["train"],
    args.dataset_file["test"],
    args.dataset_file["types_path"]
]

print("Checking required dataset files:")
all_files_exist = True
for file_path in required_files:
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
        print(f"✓ {file_path} ({file_size:.2f} MB)")
    else:
        print(f"❌ {file_path} - NOT FOUND")
        all_files_exist = False

if all_files_exist:
    print("\n✅ All required files are present!")
else:
    print("\n❌ Some required files are missing. Please check your dataset upload.")

## Step 9: Start Training

In [ ]:
# Initialize trainer
trainer = D2E2S_Trainer(args)

print("🚀 Starting D2E2S training with optimal hyperparameters...")
print(f"Configuration: {OPTIMAL_CONFIG}")
print(f"Expected F1 score: ~0.8644 (based on optimization results)")

# Start training
trained_model = trainer.train_model(
    train_path=args.dataset_file["train"],
    test_path=args.dataset_file["test"],
    types_path=args.dataset_file["types_path"],
    input_reader_cls=JsonInputReader,
)

print(f"\n🎉 Training completed successfully!")
print(f"Final best F1 score: {trainer.max_pair_f1:.4f} at epoch {trainer.best_epoch}")

## Step 10: Save Results and Model

In [ ]:
# Save training results
results = {
    "dataset": args.dataset,
    "best_f1_score": float(trainer.max_pair_f1),
    "best_epoch": trainer.best_epoch,
    "total_epochs": args.epochs,
    "hyperparameters": OPTIMAL_CONFIG,
    "model_parameters": sum(p.numel() for p in trained_model.parameters()),
}

# Save results to JSON
with open('/kaggle/working/training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("📊 Training Results:")
print(f"  Dataset: {results['dataset']}")
print(f"  Best F1 Score: {results['best_f1_score']:.4f}")
print(f"  Best Epoch: {results['best_epoch']}")
print(f"  Total Epochs: {results['total_epochs']}")
print(f"  Model Parameters: {results['model_parameters']:,}")

# Save model checkpoint
model_save_path = '/kaggle/working/d2e2s_best_model.pth'
torch.save({
    'model_state_dict': trained_model.state_dict(),
    'config': args,
    'results': results
}, model_save_path)

print(f"\n💾 Model saved to: {model_save_path}")
print(f"📄 Results saved to: /kaggle/working/training_results.json")
print(f"📋 Detailed logs saved to: {args.log_path}")

## Step 11: Performance Analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Read training logs if available
try:
    log_files = [f for f in os.listdir(args.log_path) if f.endswith('.csv')]
    if log_files:
        print("📈 Training logs found:")
        for log_file in log_files:
            print(f"  - {log_file}")
            
        # Try to plot training progress
        train_log = None
        for log_file in log_files:
            if 'train' in log_file.lower():
                train_log = pd.read_csv(os.path.join(args.log_path, log_file))
                break
                
        if train_log is not None and 'loss_avg' in train_log.columns:
            plt.figure(figsize=(12, 4))
            
            plt.subplot(1, 2, 1)
            plt.plot(train_log['loss_avg'])
            plt.title('Training Loss')
            plt.xlabel('Iteration')
            plt.ylabel('Average Loss')
            plt.grid(True)
            
            plt.subplot(1, 2, 2)
            if 'lr' in train_log.columns:
                plt.plot(train_log['lr'])
                plt.title('Learning Rate Schedule')
                plt.xlabel('Iteration')
                plt.ylabel('Learning Rate')
                plt.grid(True)
            
            plt.tight_layout()
            plt.savefig('/kaggle/working/training_progress.png', dpi=150, bbox_inches='tight')
            plt.show()
            
            print("📊 Training progress plot saved to: /kaggle/working/training_progress.png")
        
except Exception as e:
    print(f"Could not generate training plots: {e}")

# Summary
print("\n" + "="*60)
print("🎯 TRAINING SUMMARY")
print("="*60)
print(f"Model: D2E2S (Dual-channel Enhanced Entity-Sentiment)")
print(f"Dataset: {args.dataset} (Restaurant Reviews 2015)")
print(f"Configuration: Hybrid GCN with optimal hyperparameters")
print(f"Best F1 Score: {trainer.max_pair_f1:.4f}")
print(f"Achieved at Epoch: {trainer.best_epoch}")
print(f"Total Training Time: {args.epochs} epochs")
print(f"Device Used: {args.device}")
print("\n✅ Training completed successfully!")
print("📁 All outputs saved to /kaggle/working/")

## 🎉 Training Complete!

### What was accomplished:
1. ✅ Installed all required dependencies
2. ✅ Set up optimal hyperparameters (F1: 0.8644)
3. ✅ Loaded and preprocessed the 15res dataset
4. ✅ Trained D2E2S model with Hybrid GCN configuration
5. ✅ Saved model checkpoint and training results
6. ✅ Generated performance analysis

### Output Files:
- `d2e2s_best_model.pth` - Trained model checkpoint
- `training_results.json` - Training metrics and configuration
- `training_progress.png` - Training loss and learning rate plots
- `log/` directory - Detailed training logs

### Key Results:
- **Model**: D2E2S with Hybrid GCN
- **Dataset**: 15res (Restaurant Reviews 2015)
- **Expected F1**: ~0.8644 (based on hyperparameter optimization)
- **Configuration**: Production-ready optimal parameters

The model is now ready for inference on aspect-based sentiment analysis tasks!